In [1]:
import random
import os
import matplotlib.pyplot as plt
import argparse
from utils import *
from models.lstm_classifier import LSTMClassifier
from models.gru_classifier import GRUClassifier
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [19]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
train = load_data('data/utterances.train')
val = load_data('data/utterances.valid')

In [4]:
train = pd.concat([train, val]).sample(frac=1) # Merge train & val then shuffle
val_split = len(val) / len(train)
val_split

0.09237789951132092

In [5]:
test = load_data('data/utterances.test', train=False)

In [7]:
classes = train['dialog_act'].unique()
num_classes = len(classes)
max_len = train['utterance_t'].apply(len).max()

In [9]:
(X_train, y_train), (vocab, word_to_idx, idx_to_word) = process_data(train, max_len)

In [10]:
test_vocab = [word for row in test['utterance_t'] for word in row if word not in vocab]
for word in test_vocab:
    idx = len(word_to_idx)
    word_to_idx[word] = idx
    idx_to_word[idx] = word
    vocab.add(word)

In [11]:
f = open('/projekte/slu/share/emb/glove.twitter.27B.50d.txt', 'r', encoding='utf-8')
weights = {}
for line in f.readlines():
    line = line.split()
    weights[line[0]] = np.array(line[1:], dtype='float32')
f.close()
del weights['0.45973'] # Corrects an error in the file

In [12]:
num_dims = weights[random.choice(list(weights.keys()))].shape[0]
add_unknown_words(weights, word_to_idx, k=num_dims, min_df=0)
W, word_idx = get_W(weights, k=num_dims)

In [ ]:
# embds_path = '/projekte/slu/share/GoogleNews-vectors-negative300.bin' 
# w2v = load_bin_vec(embds_path, word_to_idx)
# add_unknown_words(w2v, word_to_idx)
# W, word_idx = get_W(w2v)

In [20]:
model = LSTMClassifier(num_classes, max_len, W)
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 106, 50)           60195350  
_________________________________________________________________
lstm_2 (LSTM)                (None, 106, 128)          91648     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 31)                3999      
Total params: 60,290,997
Trainable params: 60,290,997
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
%%time
history = model.fit(X_train, y_train, validation_split=val_split * 2,
                    epochs=3, batch_size=128)

Train on 176502 samples, validate on 40000 samples
Epoch 1/3
 29312/176502 [===>..........................] - ETA: 15:46 - loss: 1.5408 - acc: 0.6135

In [ ]:
model.save('lstm_model.h5')

In [16]:
X_test = sequence.pad_sequences(test['utterance_t'].apply(process_seq, args=[word_idx]),
                                maxlen=max_len)

In [ ]:
pred = model.predict_classes(X_test)
pred

In [ ]:
len([num for num in pred if num != 1])